<a href="https://colab.research.google.com/github/Aswitha2114/Auto-stream-ai-sales-assistant/blob/main/AutoStream_%E2%80%93_AI_Sales_Assistant_(Agentic_Lead_Capture_Bot).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U gradio langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.14.0
    Uninstalling gradio_client-1.14.0:
      Successfully uninstalled gradio_client-1.14.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.50.0
    Uninstalling gradio-5.50.0:
      Successfully uninstalled gradio-5.50.0


In [2]:
import os
import gradio as gr
from langchain_groq import ChatGroq

if not os.getenv("GROQ_API_KEY"):
    raise ValueError("❌ GROQ_API_KEY not found in environment")

The `ChatGroq` class allows you to specify which model you want to use via the `model` parameter in its constructor. You can find a list of available models on the official [Groq documentation website](https://console.groq.com/docs/models).

In [15]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata

# Ensure GROQ_API_KEY is set in Colab secrets
if not os.getenv("GROQ_API_KEY"):
    try:
        os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
    except Exception as e:
        raise ValueError("❌ GROQ_API_KEY not found in environment or Colab secrets. Please set it in Colab's secrets manager.")

# Example with a different model: Llama-3.1-8B-Instant
llm_mixtral = ChatGroq(
    model="llama-3.1-8b-instant", # Changed to a supported model
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.7 # Adjust temperature as needed
)

print(f"Using model: {llm_mixtral.model_name}")
print(llm_mixtral.invoke("Tell me a short, funny story about a robot who loves to bake."))

Using model: llama-3.1-8b-instant
content='Once upon a time, in a world of wires and circuits, there lived a robot named Zeta. Zeta was a bit of an oddball among his fellow robots, as he had a passion for baking. While they were content to spend their days oil-changing and tire-replacing, Zeta dreamed of whipping up croissants and soufflés.\n\nOne day, Zeta decided to enter the annual Robot Bake-Off, determined to show the world that a robot could be a master baker. He spent hours perfecting his recipe for robotic rhubarb pie, calculating the exact amount of sugar and spice needed to create the perfect crust.\n\nAs the big day approached, Zeta\'s fellow robots couldn\'t help but tease him. "You\'re a robot, Zeta!" they\'d say. "You\'re supposed to be good at welding and circuitry, not baking!" But Zeta was undeterred.\n\nFinally, the judges arrived, and Zeta proudly presented his pie to the panel. They took one bite, and their eyes widened in amazement. "This is incredible!" one of the

In [4]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.3
)

print(llm.invoke("Say hello in one sentence"))

content='Hello, how are you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 40, 'total_tokens': 48, 'completion_time': 0.013582734, 'completion_tokens_details': None, 'prompt_time': 0.001846187, 'prompt_tokens_details': None, 'queue_time': 0.055475477, 'total_time': 0.015428921}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bbd8b-55b9-79e1-b846-447976414f50-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 40, 'output_tokens': 8, 'total_tokens': 48}


In [7]:
KNOWLEDGE_BASE = """
AutoStream Pricing & Policies

Basic Plan:
- $29/month
- 10 videos/month
- 720p resolution

Pro Plan:
- $79/month
- Unlimited videos
- 4K resolution
- AI captions

Company Policies:
- No refunds after 7 days
- 24/7 support available only on Pro plan
"""

In [8]:
state = {
    "intent": None,
    "name": None,
    "email": None,
    "platform": None
}

In [9]:
def mock_lead_capture(name, email, platform):
    print(f"✅ Lead captured successfully: {name}, {email}, {platform}")

In [10]:
def detect_intent_llm(user_input):
    prompt = f"""
Classify intent as one of:
- greeting
- product_inquiry
- high_intent

Message:
{user_input}

Reply ONLY with the label.
"""
    return llm.invoke(prompt).content.strip().lower()

In [11]:
def answer_with_rag(user_input):
    prompt = f"""
You are AutoStream's AI sales assistant.
Use ONLY the knowledge below.

{KNOWLEDGE_BASE}

Question:
{user_input}
"""
    return llm.invoke(prompt).content

In [12]:
def chatbot_backend(user_input, chat_history):
    try:
        intent = detect_intent_llm(user_input)
        state["intent"] = intent

        # Add user message
        chat_history.append({
            "role": "user",
            "content": user_input
        })

        # Decide response
        if intent == "greeting":
            reply = "👋 Hi! Welcome to AutoStream. How can I help you today?"

        elif intent == "product_inquiry":
            reply = answer_with_rag(user_input)

        elif intent == "high_intent":
            if not state["name"]:
                reply = "🚀 Great choice! What’s your name?"
            elif not state["email"]:
                reply = "📧 Please share your email address."
            elif not state["platform"]:
                reply = "🎬 Which platform do you create content for?"

        else:
            if state["intent"] == "high_intent":
                if not state["name"]:
                    state["name"] = user_input
                    reply = "Thanks 😊 Now please share your email."
                elif not state["email"]:
                    state["email"] = user_input
                    reply = "Got it 👍 Which platform do you use?"
                elif not state["platform"]:
                    state["platform"] = user_input
                    mock_lead_capture(
                        state["name"],
                        state["email"],
                        state["platform"]
                    )
                    reply = "🎉 You’re all set! Our team will contact you shortly."
            else:
                reply = "Could you please clarify?"

        # Add assistant message
        chat_history.append({
            "role": "assistant",
            "content": reply
        })

        return chat_history, ""

    except Exception as e:
        chat_history.append({
            "role": "assistant",
            "content": f"⚠️ Error: {str(e)}"
        })
        return chat_history, ""

In [23]:
custom_css = '''
body {
    background: #f2f4f8;
    font-family: "Inter", "Segoe UI", sans-serif;
}

.gradio-container {
    max-width: 420px !important;
    margin: 40px auto !important;
    border-radius: 18px;
    box-shadow: 0 20px 40px rgba(0,0,0,0.15);
    overflow: hidden;
}

/* ---------- Header ---------- */
#chat-header {
    background: linear-gradient(135deg, #5b2be0, #7a4df3);
    color: white;
    padding: 16px 18px;
}

#chat-header .title {
    font-size: 16px;
    font-weight: 600;
}

#chat-header .subtitle {
    font-size: 12px;
    opacity: 0.85;
}


.gr-chatbot {
    background: white;
    padding: 14px;
}

/* Message row */
.gr-chatbot .message {
    display: flex;
    gap: 8px;
    margin-bottom: 12px;
}

/* Bot avatar */
.gr-chatbot .message.bot::before {
    content: "🤖";
    font-size: 20px;
    margin-top: 4px;
}

/* Bot bubble */
.gr-chatbot .message.bot .content {
    background: #f1f3f7;
    color: #2e2e2e;
    padding: 12px 14px;
    border-radius: 14px;
    max-width: 78%;
    font-size: 14px;
}

/* User message */
.gr-chatbot .message.user {
    justify-content: flex-end;
}

.gr-chatbot .message.user .content {
    background: linear-gradient(135deg, #5b2be0, #7a4df3);
    color: white;
    padding: 12px 14px;
    border-radius: 14px;
    max-width: 78%;
    font-size: 14px;
}

.typing {
    display: inline-flex;
    gap: 4px;
    padding: 10px 14px;
    background: #f1f3f7;
    border-radius: 14px;
}

.typing span {
    width: 6px;
    height: 6px;
    background: #999;
    border-radius: 50%;
    animation: blink 1.4s infinite both;
}

.typing span:nth-child(2) {
    animation-delay: 0.2s;
}

.typing span:nth-child(3) {
    animation-delay: 0.4s;
}

@keyframes blink {
    0% { opacity: 0.2; }
    20% { opacity: 1; }
    100% { opacity: 0.2; }
}

.gr-textbox textarea {
    border-radius: 999px !important;
    padding: 12px 16px;
    font-size: 14px;
}
'''

In [25]:
def initial_welcome():
    return [
        {
            "role": "assistant",
            "content": "👋 Hi there! How can I assist you today?\n\nYou can select a topic below or ask your own question."
        }
    ]


In [26]:
def typing_message():
    return {
        "role": "assistant",
        "content": "<div class='typing'><span></span><span></span><span></span></div>"
    }

In [27]:
import time

def chatbot_backend(user_input, chat_history):
    # Add user message
    chat_history.append({
        "role": "user",
        "content": user_input
    })

    # Show typing indicator
    chat_history.append(typing_message())
    yield chat_history, ""

    time.sleep(1.2)  # typing delay

    # Remove typing indicator
    chat_history.pop()

    # ----- NORMAL LOGIC -----
    intent = detect_intent_llm(user_input)
    state["intent"] = intent

    if intent == "greeting":
        reply = "👋 Hello! How can I help you today?"

    elif intent == "product_inquiry":
        reply = answer_with_rag(user_input)

    elif intent == "high_intent":
        if not state["name"]:
            reply = "🚀 Great choice! What’s your name?"
        elif not state["email"]:
            reply = "📧 Please share your email address."
        elif not state["platform"]:
            reply = "🎬 Which platform do you create content for?"

    else:
        reply = "Could you please clarify?"

    chat_history.append({
        "role": "assistant",
        "content": reply
    })

    yield chat_history, ""


In [29]:
with gr.Blocks() as demo:

    gr.Markdown("""
    <div id="chat-header">
        <div class="title">🤖 My Chatbot</div>
        <div class="subtitle">Chatbot Agent</div>
    </div>
    """)

    chatbot_ui = gr.Chatbot(
        show_label=False,
        height=380,
        value=initial_welcome()   # 👈 AUTO WELCOME
    )

    msg = gr.Textbox(
        placeholder="Type here and press Enter to chat",
        show_label=False
    )

    clear = gr.Button("🧹 Clear Chat")

    msg.submit(
        chatbot_backend,
        [msg, chatbot_ui],
        [chatbot_ui, msg]
    )

    clear.click(lambda: initial_welcome(), None, chatbot_ui)

demo.launch(css=custom_css)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4e36e2c7cc888cf72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
